In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np
from scipy.special import gamma
from scipy import signal
import time
import itertools
from itertools import combinations
try:
    from tqdm import tqdm_notebooks as tqdm
except ImportError:
    tqdm = lambda x: x
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
###
from collections import defaultdict

from scipy.stats import pearsonr, zscore
from MFDFA import fgn
# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()
from tqdm import tqdm


import matplotlib as mpl
from matplotlib import cm

import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')
import shogun as sg
from shogun import *
from scipy.stats import norm, laplace
import random
import shogunMMDutils as mmdutils

# import stylised_facts_data_utilities.createLOB as createLOB
# import stylised_facts_data_utilities.gpyNARX as gpyNARX
# import stylised_facts_data_utilities.longtaildistr as longtail
%matplotlib inline

plt.style.use(os.path.join(mpl.get_configdir(),'latexstyle.mplstyle'))

In [2]:
mmd_path ='/media/ak/My Passport/Experiment Data/TwoSampleExperiments/MicroPriceExperiments'

In [3]:
mmd_experiment_files =os.listdir(mmd_path)

In [4]:
mmd_experiment_files

['DU11_lagVolumeBarDf_best_p_values.pkl',
 'DU11_lagVolumeBarDf_permutations.pkl',
 'DU11_lagVolumeBarDf_rejections.pkl',
 'DU1MicroPriceMMD_permutations.pkl',
 'DU1_Rejections.pkl',
 'FV12_lagVolumeBarDf_best_p_values.pkl',
 'FV12_lagVolumeBarDf_permutations.pkl',
 'FV12_lagVolumeBarDf_rejections.pkl',
 'FV1_permutation_tests.pkl',
 'FV1_Rejections.pkl',
 'G_1_permutation_tests.pkl',
 'G_1_Rejections.pkl']

In [42]:
idx =11
mmd_file= os.path.join(mmd_path, sorted(mmd_experiment_files)[idx])
mmd_file

'/media/ak/My Passport/Experiment Data/TwoSampleExperiments/MicroPriceExperiments/G_1_permutation_tests.pkl'

In [43]:
mmd_file_loaded = pickle.load(open(mmd_file, 'rb'))
mmd_file_loaded

{(0, 1): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 2): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 3): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 4): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 5): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 6): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 7): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 8): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 9): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 10): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 11): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 12): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 13): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 14): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 15): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 16): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 17): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 18): (0.0, 0.05, 0.0, 0.05, 0.0),
 (0, 19): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 2): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 3): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 4): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 5): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 6): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 7): (0.0, 0.05, 0.0, 0.05, 0.0),
 (1, 8): (0.0, 0.05, 0.0, 0.05, 0.0),
 (

In [14]:
def power(dict_results): 
    power = {}
    rejections = {}

    for key in dict_results.keys():
        #print(key)
        rejections[key] = 0
        for i in range(len(dict_results[key])):
            if dict_results[key][i][0] > dict_results[key][i][1]:
                rejections[key] += 1
            else:
                pass
        # statistical power
        power[key] = 100 * rejections[key] / len(dict_results[key])
    
    power_df = pd.DataFrame.from_dict(data=power, orient='index').reset_index()
    power_df0 = power_df['index'].apply(pd.Series)
    power_df1 = pd.concat([power_df0, power_df], axis=1, ignore_index=True).drop([4], axis=1)
    power_df2 = power_df1.rename(columns={0: 'dimension', 1: 'sample_size', 2: 'lag', 3: 'test', 5: 'power'}).sort_values(by=['sample_size', 'lag', 'test'], axis=0)
   
    return power_df2

In [15]:
power(dict_results)

,dimension,sample_size,lag,test
0,1,2000,1,"(1, 2000, 1)"
1,1,2000,2,"(1, 2000, 2)"
2,1,2000,3,"(1, 2000, 3)"


In [16]:
# confidence intervals
def confidence(mean):
    interval = 1.96 * np.sqrt((mean * (100 - mean)) / 100)
    return interval

In [17]:
def power_mmd(dict_results):
    power = {}
    rejections = {}

    for key in dict_results.keys():
        #print(key)
        rejections[key] = 0
        for i in range(len(dict_results[key])):
            if dict_results[key][i][0] > dict_results[key][i][1]:
                rejections[key] += 1
            else:
                pass
        # statistical power
        power[key] = 100 * rejections[key] / len(dict_results[key])
    
    power_df = pd.DataFrame.from_dict(data=power, orient='index').reset_index()
    power_df0 = power_df['index'].apply(pd.Series)
    power_df1 = pd.concat([power_df0, power_df], axis=1, ignore_index=True).drop([4], axis=1)
    power_df2 = power_df1.rename(columns={0: 'dimension', 1: 'sample_size', 2: 'delta', 5: 'power'}).sort_values(by=['sample_size', 'delta'], axis=0)
    
    return power_df2

In [18]:
power_mmd(dict_results)

,dimension,sample_size,delta,3
0,1,2000,1,"(1, 2000, 1)"
1,1,2000,2,"(1, 2000, 2)"
2,1,2000,3,"(1, 2000, 3)"


In [25]:
power_df2 = power_mmd(dict_results)

sample_2000 = power_df2[power_df2['sample_size']==2000]

In [26]:
sample_2000

,dimension,sample_size,delta,3
0,1,2000,1,"(1, 2000, 1)"
1,1,2000,2,"(1, 2000, 2)"
2,1,2000,3,"(1, 2000, 3)"


In [28]:
sample_2000['delta']


0    1
1    2
2    3
Name: delta, dtype: int64